In [ ]:
import os
import plotly.offline as py
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import numpy
import plotly.io as pio
import scipy.optimize as optimize
# colorbar
colorscale=[
        [0, "#ff0000"],
        [0.1, "#ff3300"],
        [0.2, "#ff6600"],
        [0.3, "#ff9900"],
        [0.4, "#ffcc00"],
        [0.5, "#ffff00"],
        [0.6, "#cdee0a"],
        [0.7, "#9add14"],
        [0.8, "#67cc1e"],
        [0.9, "#34bb28"],
        [1, "#00a933"],]
colorbar=dict(tickmode='array',tickvals=[0.0, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],ticktext=['0 %','10 %','20 %','30 %','40 %','50 %','60 %','70 %','80 %','90 %','100 %'],len=1.05,ticklabeloverflow='allow',outlinewidth=0,ticks='inside')

filename='vdi4657_chapter_9-2-3-1_results.csv'

In [ ]:
# ONLY NECCECARY DIRECTLY AFTER THE PARAMETER-VARIATION
# read raw results and save

#df=pd.DataFrame()
#for folder in os.listdir('results'):
#    try:
#        df=pd.concat([df,pd.read_csv(r'results/'+folder+'/results_kpi.csv')])
#    except:
#        print('No `results_kpi.csv` in ' + folder)
#df.sort_values('e_bat')
#df.sort_values('p_pv')
#df.to_csv(filename)

In [ ]:
#Fit-Functions
guess = (1.0,1.0,1.0,1.0)
def func_a(data, k1,k2,k3,k4):
    return numpy.maximum((k1*data[0]+k2)*(k4*data[1]+1)/((data[0]+k3)*(data[1]+1)),0)
def func_e(data, k1,k2,k3,k4):
    return numpy.minimum((k1*data[0]+k2)*(k4*data[1]+1)/((data[0]+k3)*(data[1]+1)),1)

In [ ]:
df=pd.read_csv(filename)
for profile in df['electric_loadprofile'].unique():
    # Autarkiegrad
    # load results and fit
    df=pd.read_csv(filename)
    # Filter data for fit
    df=df.loc[df['electric_loadprofile']==profile,:]
    df=df.loc[df['p_pv']>0,:]
    xData = df['p_pv'].values
    yData = df['e_bat'].values
    zData = df['autarkiegrad'].values
    # place the data in a single list
    data = [xData, yData, zData]
    params_a, pcov = optimize.curve_fit(func_a, [xData,yData], zData, guess)

    # calc values with fitting functions
    df1=pd.DataFrame()
    PV = list(numpy.arange(0.25,2.75,0.25))
    BAT = list(numpy.arange(0,2.75,0.25))
    e_bat=[]
    p_pv=[]
    eigen=[]
    aut=[]
    for bat in BAT:
        for pv in PV:
            e_bat.append(bat)
            p_pv.append(pv)
            aut.append(func_a([pv,bat],params_a[0],params_a[1],params_a[2],params_a[3]))
    df1['e_bat']=e_bat
    df1['p_pv']=p_pv
    df1['autarkiegrad']=aut
    fig= go.Figure()
    # Filter for plot
    df=df.loc[df['p_pv']>=0.25,:]
    df=df.loc[df['e_bat']!=0.05,:]
    df=df.loc[df['e_bat']!=0.10,:]
    df=df.loc[df['e_bat']!=0.15,:]
    df=df.loc[df['e_bat']!=0.20,:]
    fig.add_trace(go.Heatmap(x=df['p_pv'].values,
                                    y=df['e_bat'].values,
                                    z=df['autarkiegrad'].values,
                                    text=(df['autarkiegrad']*100).round(0),
                                    texttemplate="%{text} %",
                                    textfont={"size":10,'color':'black'},
                                    coloraxis='coloraxis'))
    for i in range(11):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(12):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.5, dash='dot'),
                                showlegend=False))
    fig.update_layout(width=600,height=600,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
    fig.update_layout(
        title={
            'text': "$ \Large \mathsf{Autarkiegrad}$",
            'font':{'color':'black'},
            'y':0.875,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(range=[-0.13,2.63],tick0=0,dtick=0.25,tickangle=270,title={'text':'$ c_{\mathsf{S,eff,sp}} \ \mathsf{in \ kWh/MWh}$','standoff':5,'font':{'color':'black'},})
    fig.update_xaxes(range=[0.12,2.63],tick0=0,dtick=0.25,title={'text':'$ p_{\mathsf{PV,sp}} \ \mathsf{in \ kWp/MWh}$','standoff':5,'font':{'color':'black'},})
    #fig.show()
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-sim.png', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-sim.svg', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-sim.pdf', scale=2)
    fig=go.Figure()
    fig.add_trace(go.Heatmap(x=df1['p_pv'].values,
                                    y=df1['e_bat'].values,
                                    z=df1['autarkiegrad'].values,
                                    text=(df1['autarkiegrad']*100).round(),
                                    texttemplate="%{text} %",
                                    textfont={"size":10,'color':'black'},
                                    coloraxis='coloraxis',
                                    opacity=1,))
    for i in range(10):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(11):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                showlegend=False))
    for i in range(11):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(12):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.5, dash='dot'),
                                showlegend=False))
    fig.update_layout(width=600,height=600,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
    fig.update_layout(
        title={
            'text': "$ \Large \mathsf{Autarkiegrad}$",
            'font':{'color':'black'},
            'y':0.875,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(range=[-0.13,2.63],tick0=0,dtick=0.25,tickangle=270,title={'text':'$ c_{\mathsf{S,eff,sp}} \ \mathsf{in \ kWh/MWh}$','standoff':5,'font':{'color':'black'},})
    fig.update_xaxes(range=[0.12,2.63],tick0=0,dtick=0.25,title={'text':'$ p_{\mathsf{PV,sp}} \ \mathsf{in \ kWp/MWh}$','standoff':5,'font':{'color':'black'},})
    #fig.show()
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-fit.png', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-fit.svg', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-fit.pdf', scale=2)


    print('fitted parameters', params_a)
    modelPredictions = func_a(data, *params_a) 
    absError = modelPredictions - zData
    SE = numpy.square(absError) # squared errors
    MSE = numpy.mean(SE) # mean squared errors
    RMSE_A = numpy.sqrt(MSE) # Root Mean Squared Error, RMSE
    Rsquared = 1.0 - (numpy.var(absError) / numpy.var(zData))
    print('RMSE:', RMSE_A)
    print('R-squared:', Rsquared)
    df1=df1.sort_values(['e_bat','p_pv'])
    df1['a_dif']=df1['autarkiegrad'].values-df.sort_values(['e_bat','p_pv'])['autarkiegrad'].values
    fig=go.Figure()
    fig.add_trace(go.Heatmap(x=df1['p_pv'].values,
                                    y=df1['e_bat'].values,
                                    z=df1['a_dif'].values,
                                    text=(df1['a_dif']*100).round(1),
                                    texttemplate="%{text} %",
                                    textfont={"size":10,'color':'black'},
                                    coloraxis='coloraxis'))
    for i in range(11):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(12):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.5, dash='dot'),
                                showlegend=False))
    fig.update_layout(width=600,height=600,coloraxis = {'cmin':-0.07,'cmax':0.07},)
    fig.update_layout(
        title={
            'text': "$ \mathsf{Autarkiegradsdifferenz \ Fit \ zu \ Simulation}$",
            'font':{'color':'black'},
            'y':0.875,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(range=[-0.13,2.63],tick0=0,dtick=0.25,tickangle=270,title={'text':'$ c_{\mathsf{S,eff,sp}} \ \mathsf{in \ kWh/MWh}$','standoff':5,'font':{'color':'black'},})
    fig.update_xaxes(range=[0.12,2.63],tick0=0,dtick=0.25,title={'text':'$ p_{\mathsf{PV,sp}} \ \mathsf{in \ kWp/MWh}$','standoff':5,'font':{'color':'black'},})
    #fig.show()
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-dif.png', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-dif.svg', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_a-dif.pdf', scale=2)

    # Eigenverbrauchsanteil
    # load results and fit

    df=pd.read_csv(filename)
    # Filter data for fit
    df=df.loc[df['electric_loadprofile']==profile,:]
    df=df.loc[df['p_pv']>0,:]
    df=df.loc[df['eigenverbrauch']<1,:]
    xData = df['p_pv'].values
    yData = df['e_bat'].values
    zData = df['eigenverbrauch'].values
    # place the data in a single list
    data = [xData, yData, zData]
    params_e, pcov = optimize.curve_fit(func_e, [xData,yData], zData, guess)

    # calc values with fitting functions
    df1=pd.DataFrame()
    PV = list(numpy.arange(0.25,2.75,0.25))
    BAT = list(numpy.arange(0,2.75,0.25))
    e_bat=[]
    p_pv=[]
    eigen=[]
    for bat in BAT:
        for pv in PV:
            e_bat.append(bat)
            p_pv.append(pv)
            eigen.append(func_e([pv,bat],params_e[0],params_e[1],params_e[2],params_e[3]))
    df1['e_bat']=e_bat
    df1['p_pv']=p_pv
    df1['eigenverbrauch']=eigen
    fig= go.Figure()
    # Filter for plot
    df=df.loc[df['p_pv']>=0.25,:]
    df=df.loc[df['e_bat']!=0.05,:]
    df=df.loc[df['e_bat']!=0.10,:]
    df=df.loc[df['e_bat']!=0.15,:]
    df=df.loc[df['e_bat']!=0.20,:]
    fig.add_trace(go.Heatmap(x=df['p_pv'].values,
                                    y=df['e_bat'].values,
                                    z=df['eigenverbrauch'].values,
                                    text=(df['eigenverbrauch']*100).round(0),
                                    texttemplate="%{text} %",
                                    textfont={"size":10,'color':'black'},
                                    coloraxis='coloraxis'))
    for i in range(11):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(12):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.5, dash='dot'),
                                showlegend=False))
    fig.update_layout(width=600,height=600,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
    fig.update_layout(
        title={
            'text': "$ \Large \mathsf{Eigenverbrauch}$",
            'font':{'color':'black'},
            'y':0.875,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(range=[-0.13,2.63],tick0=0,dtick=0.25,tickangle=270,title={'text':'$ c_{\mathsf{S,eff,sp}} \ \mathsf{in \ kWh/MWh}$','standoff':5,'font':{'color':'black'},})
    fig.update_xaxes(range=[0.12,2.63],tick0=0,dtick=0.25,title={'text':'$ p_{\mathsf{PV,sp}} \ \mathsf{in \ kWp/MWh}$','standoff':5,'font':{'color':'black'},})
    #fig.show()
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-sim.png', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-sim.svg', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-sim.pdf', scale=2)

    fig=go.Figure()
    fig.add_trace(go.Heatmap(x=df1['p_pv'].values,
                                    y=df1['e_bat'].values,
                                    z=df1['eigenverbrauch'].values,
                                    text=(df1['eigenverbrauch']*100).round(0),
                                    texttemplate="%{text} %",
                                    textfont={"size":10,'color':'black'},
                                    coloraxis='coloraxis',
                                    opacity=1,))

    for i in range(11):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(12):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.5, dash='dot'),
                                showlegend=False))
    fig.update_layout(width=600,height=600,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
    fig.update_layout(
        title={
            'text': "$ \Large \mathsf{Eigenverbrauch}$",
            'font':{'color':'black'},
            'y':0.875,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(range=[-0.13,2.63],tick0=0,dtick=0.25,tickangle=270,title={'text':'$ c_{\mathsf{S,eff,sp}} \ \mathsf{in \ kWh/MWh}$','standoff':5,'font':{'color':'black'},})
    fig.update_xaxes(range=[0.12,2.63],tick0=0,dtick=0.25,title={'text':'$ p_{\mathsf{PV,sp}} \ \mathsf{in \ kWp/MWh}$','standoff':5,'font':{'color':'black'},})
    #fig.show()
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-fit.png', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-fit.svg', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-fit.pdf', scale=2)
    print('fitted parameters', params_e)
    modelPredictions = func_e(data, *params_e) 
    absError = modelPredictions - zData
    SE = numpy.square(absError) # squared errors
    MSE = numpy.mean(SE) # mean squared errors
    RMSE_E = numpy.sqrt(MSE) # Root Mean Squared Error, RMSE
    Rsquared = 1.0 - (numpy.var(absError) / numpy.var(zData))
    print('RMSE:', RMSE_E)
    print('R-squared:', Rsquared)
    df1=df1.sort_values(['e_bat','p_pv'])
    df1['e_dif']=df1['eigenverbrauch'].values-df.sort_values(['e_bat','p_pv'])['eigenverbrauch'].values
    fig=go.Figure()
    fig.add_trace(go.Heatmap(x=df1['p_pv'].values,
                                    y=df1['e_bat'].values,
                                    z=df1['e_dif'].values,
                                    text=(df1['e_dif']*100).round(1),
                                    texttemplate="%{text} %",
                                    textfont={"size":10,'color':'black'},
                                    coloraxis='coloraxis'))
    for i in range(11):
        fig.add_trace(go.Scatter(x=[i/4+0.125, i/4+0.125],
                                y=[-0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.25, dash='dot'),
                                
                                showlegend=False))
    for i in range(12):
        fig.add_trace(go.Scatter(y=[i/4-0.125, i/4-0.125],
                                x=[0.125, 2.625],
                                mode="lines",
                                line=dict(color='black', width=0.5, dash='dot'),
                                showlegend=False))
    fig.update_layout(width=600,height=600,coloraxis = {'cmin':-0.07,'cmax':0.07},)
    fig.update_layout(
        title={
            'text': "$ \mathsf{Eigenverbrauchsdifferenz \ Fit \ zu \ Simulation}$",
            'font':{'color':'black'},
            'y':0.875,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.update_yaxes(range=[-0.13,2.63],tick0=0,dtick=0.25,tickangle=270,title={'text':'$ c_{\mathsf{S,eff,sp}} \ \mathsf{in \ kWh/MWh}$','standoff':5,'font':{'color':'black'},})
    fig.update_xaxes(range=[0.12,2.63],tick0=0,dtick=0.25,title={'text':'$ p_{\mathsf{PV,sp}} \ \mathsf{in \ kWp/MWh}$','standoff':5,'font':{'color':'black'},})
    #fig.show()
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-dif.png', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-dif.svg', scale=2)
    pio.write_image(fig, 'vdi4657_chapter_9-2-3/'+profile+'_pv_4_average_2015_e-dif.pdf', scale=2)
    df_para=pd.DataFrame()
    df_para['Lastprofilgruppe']=[profile,profile]
    df_para['Parameter']=['k_e','k_a']
    df_para['1']=[params_e[0],params_a[0]]
    df_para['2']=[params_e[1],params_a[1]]
    df_para['3']=[params_e[2],params_a[2]]
    df_para['4']=[params_e[3],params_a[3]]
    df_para['RMSE']=[RMSE_E,RMSE_A]
    df_para.to_csv('vdi4657_chapter_9-2-3/'+profile+'_para.csv',index=False)